# **Machine Learning of Ecologic Variables (Random Forest)**

In [1]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import zip_longest

# Data
df = pd.read_csv('AVONETplusClim.csv')
df_clean = df.copy()
df_clean = df_clean[(df_clean['Mass'] > 0) & (df_clean['Mass'] < 11500)]
df_clean = df_clean[(df_clean['Tail.Length'] > 0.1) & (df_clean['Tail.Length'] < 535)]
df_clean = df_clean[(df_clean['Tarsus.Length'] > 0) & (df_clean['Tarsus.Length'] < 350)]
df_clean = df_clean[(df_clean['Wing.Length'] > 0.1) & (df_clean['Wing.Length'] < 650)]
df_clean = df_clean[df_clean['Hand-Wing.Index'] > 3]
df_clean['Log_Mass'] = np.log10(df_clean['Mass'])
df_clean['Log_Tail'] = np.log10(df_clean['Tail.Length'])
df_clean['Log_Tarsus'] = np.log10(df_clean['Tarsus.Length'])
df_clean['Log_Beak'] = np.log10(df_clean['Beak.Length_Culmen'])
df_clean['Log_Wing'] = np.log10(df_clean['Wing.Length'])
df_clean['Log_HWI'] = np.log10(df_clean['Hand-Wing.Index'])

## **1. Body Size (Mass)**

In [2]:
target = "Log_Mass"
predictors = ['Trophic.Niche', 'Primary.Lifestyle', 'Habitat', 'Migration']

X = pd.get_dummies(df_clean[predictors], drop_first=False)
y = df_clean[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42
)
rf.fit(X_train, y_train)
score = r2_score(y_test, rf.predict(X_test))

print("="*145)
print(f"R² Score: {score:.3f}")
print("="*145)

display_columns = []
name_width = 21
val_width = 12

for col in predictors:
    grouped = df_clean.groupby(col)['Mass'].mean().sort_values(ascending=False)

    formatted_rows = [f"{cat[:name_width]:<{name_width}} {val:>{val_width}.0f}" for cat, val in grouped.items()]
    display_columns.append(formatted_rows)

header_row = " | ".join([f"{col:<{name_width}} {'Mass (g)':>{val_width}}" for col in predictors])

print(header_row)
print("-" * len(header_row))

block_width = name_width + 1 + val_width

for row in zip_longest(*display_columns, fillvalue=" " * block_width):
    print(" | ".join(row))

print("="*145)

R² Score: 0.536
Trophic.Niche             Mass (g) | Primary.Lifestyle         Mass (g) | Habitat                   Mass (g) | Migration                 Mass (g)
-------------------------------------------------------------------------------------------------------------------------------------------------
Scavenger                     3498 | Aquatic                       1529 | Aquatic                        758 | Partial                        342
Herbivore aquatic             1562 | Terrestrial                    379 | Open                           274 | Migratory                      323
Herbivore terrestrial         1091 | Generalist                     185 | Closed                         122 | Sedentary                      161
Aquatic predator               773 | Aerial                         138 |                                    |                                   
Vertivore                      737 | Insessorial                     91 |                                   

In [3]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["Trophic Niche", "Primary Lifestyle", "Habitat", "Migration Behavior"],
    horizontal_spacing=0.15,
    vertical_spacing=0.15
)

importances = pd.Series(rf.feature_importances_, index=X.columns)

def add_panel(prefix, row, col, bar_color):
    subset = importances[importances.index.str.contains(prefix)].copy()

    subset.index = subset.index.str.replace(f"{prefix}_", "").str.replace(f"{prefix}.", "")
    
    subset = subset.sort_values(ascending=True)
    
    fig.add_trace(
        go.Bar(
            x=subset.values,
            y=subset.index,
            orientation='h',
            marker=dict(color=bar_color),
            name=prefix
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text="Importance (R²)", row=row, col=col)

add_panel("Trophic.Niche", 1, 1, "#BF616A")
add_panel("Primary.Lifestyle", 1, 2, "#8FBCBB")
add_panel("Habitat", 2, 1, "#EBCB8B")
add_panel("Migration", 2, 2, "#81A1C1")

fig.update_layout(
    title_text="Ecological Drivers of Body Mass",
    title_x=0.5,
    height=800,
    width=1100,
    showlegend=False,
    template="plotly_white"
)

fig.show()

The Random Forest model explains approximately 53% of the variation in avian body mass (R² = 0.53).

* Trophic Niche: The major predictor of body mass. Scavengers have the highest average body mass, while Nectarivores and Invertivores are the strongest predictors due to their consistently low body mass.
* Primary Lifestyle: the Insessorial (perching) category is the single strongest predictor overall, corresponding to a consistently small size range, whereas the Aquatic lifestyle predicts significantly higher body mass.
* Habitat and Migration: Shows negligible feature importance and does not significantly predict body mass in this model.

## **2. Tail Size (Tail Length)**

In [4]:
target = "Log_Tail"
predictors = ['Trophic.Niche', 'Primary.Lifestyle', 'Habitat', 'Migration']

X = pd.get_dummies(df_clean[predictors], drop_first=False)
y = df_clean[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42
)
rf.fit(X_train, y_train)
score = r2_score(y_test, rf.predict(X_test))

print("="*145)
print(f"R² Score: {score:.3f}")
print("="*145)

display_columns = []
name_width = 21
val_width = 12

for col in predictors:
    grouped = df_clean.groupby(col)['Tail.Length'].mean().sort_values(ascending=False)

    formatted_rows = [f"{cat[:name_width]:<{name_width}} {val:>{val_width}.0f}" for cat, val in grouped.items()]
    display_columns.append(formatted_rows)

header_row = " | ".join([f"{col:<{name_width}} {'Length (mm)':>{val_width}}" for col in predictors])

print(header_row)
print("-" * len(header_row))

block_width = name_width + 1 + val_width

for row in zip_longest(*display_columns, fillvalue=" " * block_width):
    print(" | ".join(row))

print("="*145)

R² Score: 0.207
Trophic.Niche          Length (mm) | Primary.Lifestyle      Length (mm) | Habitat                Length (mm) | Migration              Length (mm)
-------------------------------------------------------------------------------------------------------------------------------------------------
Scavenger                      231 | Generalist                      90 | Aquatic                         89 | Partial                         89
Vertivore                      187 | Terrestrial                     90 | Open                            88 | Sedentary                       84
Herbivore terrestrial          133 | Aquatic                         84 | Closed                          84 | Migratory                       81
Frugivore                      109 | Insessorial                     83 |                                    |                                   
Omnivore                        98 | Aerial                          75 |                                   

In [5]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["Trophic Niche", "Primary Lifestyle", "Habitat", "Migration Behavior"],
    horizontal_spacing=0.15,
    vertical_spacing=0.15
)

importances = pd.Series(rf.feature_importances_, index=X.columns)

def add_panel(prefix, row, col, bar_color):
    subset = importances[importances.index.str.contains(prefix)].copy()

    subset.index = subset.index.str.replace(f"{prefix}_", "").str.replace(f"{prefix}.", "")
    
    subset = subset.sort_values(ascending=True)
    
    fig.add_trace(
        go.Bar(
            x=subset.values,
            y=subset.index,
            orientation='h',
            marker=dict(color=bar_color),
            name=prefix
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text="Importance (R²)", row=row, col=col)

add_panel("Trophic.Niche", 1, 1, "#BF616A")
add_panel("Primary.Lifestyle", 1, 2, "#8FBCBB")
add_panel("Habitat", 2, 1, "#EBCB8B")
add_panel("Migration", 2, 2, "#81A1C1")

fig.update_layout(
    title_text="Ecological Drivers of Tail Length",
    title_x=0.5,
    height=800,
    width=1100,
    showlegend=False,
    template="plotly_white"
)

fig.show()

The Random Forest model explains approximately 23% of the variation in avian tail length (R² = 0.23).

* Trophic Niche: The major predictor of tail length. Scavengers have the longest average tail length, while Vertivores are the strongest predictors.
* Primary Lifestyle: The Insessorial category is the single strongest predictor overall, whereas the Terrestrial lifestyle predicts the longest average tail length.
* Habitat and Migration: Overall they show relatively low feature importance.

## **3. Leg Size (Tarsus Length)**

In [6]:
target = "Log_Tarsus"
predictors = ['Trophic.Niche', 'Primary.Lifestyle', 'Habitat', 'Migration']

X = pd.get_dummies(df_clean[predictors], drop_first=False)
y = df_clean[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42
)
rf.fit(X_train, y_train)
score = r2_score(y_test, rf.predict(X_test))

print("="*145)
print(f"R² Score: {score:.3f}")
print("="*145)

display_columns = []
name_width = 21
val_width = 12

for col in predictors:
    grouped = df_clean.groupby(col)['Tarsus.Length'].mean().sort_values(ascending=False)

    formatted_rows = [f"{cat[:name_width]:<{name_width}} {val:>{val_width}.0f}" for cat, val in grouped.items()]
    display_columns.append(formatted_rows)

header_row = " | ".join([f"{col:<{name_width}} {'Length (mm)':>{val_width}}" for col in predictors])

print(header_row)
print("-" * len(header_row))

block_width = name_width + 1 + val_width

for row in zip_longest(*display_columns, fillvalue=" " * block_width):
    print(" | ".join(row))

print("="*145)

R² Score: 0.619
Trophic.Niche          Length (mm) | Primary.Lifestyle      Length (mm) | Habitat                Length (mm) | Migration              Length (mm)
-------------------------------------------------------------------------------------------------------------------------------------------------
Scavenger                       87 | Aquatic                         46 | Aquatic                         49 | Partial                         34
Vertivore                       63 | Terrestrial                     42 | Open                            32 | Migratory                       31
Herbivore aquatic               55 | Generalist                      30 | Closed                          25 | Sedentary                       27
Aquatic predator                53 | Insessorial                     23 |                                    |                                   
Herbivore terrestrial           48 | Aerial                          16 |                                   

In [7]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["Trophic Niche", "Primary Lifestyle", "Habitat", "Migration Behavior"],
    horizontal_spacing=0.15,
    vertical_spacing=0.15
)

importances = pd.Series(rf.feature_importances_, index=X.columns)

def add_panel(prefix, row, col, bar_color):
    subset = importances[importances.index.str.contains(prefix)].copy()

    subset.index = subset.index.str.replace(f"{prefix}_", "").str.replace(f"{prefix}.", "")
    
    subset = subset.sort_values(ascending=True)
    
    fig.add_trace(
        go.Bar(
            x=subset.values,
            y=subset.index,
            orientation='h',
            marker=dict(color=bar_color),
            name=prefix
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text="Importance (R²)", row=row, col=col)

add_panel("Trophic.Niche", 1, 1, "#BF616A")
add_panel("Primary.Lifestyle", 1, 2, "#8FBCBB")
add_panel("Habitat", 2, 1, "#EBCB8B")
add_panel("Migration", 2, 2, "#81A1C1")

fig.update_layout(
    title_text="Ecological Drivers of Tarsus Length",
    title_x=0.5,
    height=800,
    width=1100,
    showlegend=False,
    template="plotly_white"
)

fig.show()

The Random Forest model explains approximately 58% of the variation in avian tarsus length (R² = 0.58).

* Trophic Niche: A major predictor of tarsus length. Scavengers have the longest average tarsus length, while Nectarivores are the strongest predictors due to their consistently short legs.
* Primary Lifestyle: The Terrestrial category is the single strongest predictor overall, corresponding to a requirement for long legs, whereas the Insessorial (perching) lifestyle predicts significantly shorter legs.
* Habitat and Migration: Generally shows lower feature importance, though Aquatic habitat is a significant predictor within its category, correlating with longer legs. Migration behavior shows negligible feature importance and does not significantly predict tarsus length.

## **4. Beak Size (Beak Length)**

In [8]:
target = "Log_Beak"
predictors = ['Trophic.Niche', 'Primary.Lifestyle', 'Habitat', 'Migration']

X = pd.get_dummies(df_clean[predictors], drop_first=False)
y = df_clean[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42
)
rf.fit(X_train, y_train)
score = r2_score(y_test, rf.predict(X_test))

print("="*145)
print(f"R² Score: {score:.3f}")
print("="*145)

display_columns = []
name_width = 21
val_width = 12

for col in predictors:
    grouped = df_clean.groupby(col)['Beak.Length_Culmen'].mean().sort_values(ascending=False)

    formatted_rows = [f"{cat[:name_width]:<{name_width}} {val:>{val_width}.0f}" for cat, val in grouped.items()]
    display_columns.append(formatted_rows)

header_row = " | ".join([f"{col:<{name_width}} {'Length (mm)':>{val_width}}" for col in predictors])

print(header_row)
print("-" * len(header_row))

block_width = name_width + 1 + val_width

for row in zip_longest(*display_columns, fillvalue=" " * block_width):
    print(" | ".join(row))

print("="*145)

R² Score: 0.368
Trophic.Niche          Length (mm) | Primary.Lifestyle      Length (mm) | Habitat                Length (mm) | Migration              Length (mm)
-------------------------------------------------------------------------------------------------------------------------------------------------
Scavenger                       64 | Aquatic                         59 | Aquatic                         50 | Partial                         31
Aquatic predator                62 | Terrestrial                     32 | Open                            24 | Migratory                       28
Herbivore aquatic               51 | Aerial                          23 | Closed                          23 | Sedentary                       24
Vertivore                       34 | Insessorial                     23 |                                    |                                   
Herbivore terrestrial           31 | Generalist                      22 |                                   

In [9]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["Trophic Niche", "Primary Lifestyle", "Habitat", "Migration Behavior"],
    horizontal_spacing=0.15,
    vertical_spacing=0.15
)

importances = pd.Series(rf.feature_importances_, index=X.columns)

def add_panel(prefix, row, col, bar_color):
    subset = importances[importances.index.str.contains(prefix)].copy()

    subset.index = subset.index.str.replace(f"{prefix}_", "").str.replace(f"{prefix}.", "")
    
    subset = subset.sort_values(ascending=True)
    
    fig.add_trace(
        go.Bar(
            x=subset.values,
            y=subset.index,
            orientation='h',
            marker=dict(color=bar_color),
            name=prefix
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text="Importance (R²)", row=row, col=col)

add_panel("Trophic.Niche", 1, 1, "#BF616A")
add_panel("Primary.Lifestyle", 1, 2, "#8FBCBB")
add_panel("Habitat", 2, 1, "#EBCB8B")
add_panel("Migration", 2, 2, "#81A1C1")

fig.update_layout(
    title_text="Ecological Drivers of Beak Length",
    title_x=0.5,
    height=800,
    width=1100,
    showlegend=False,
    template="plotly_white"
)

fig.show()

The Random Forest model explains approximately 31% of the variation in avian beak length (R² = 0.311).

* Trophic Niche: A major predictor of beak length. Scavengers have the longest average beaks, but Aquatic predators are the strongest predictors (highest feature importance), likely due to the strict requirement for specialized bills to catch fish. Granivores are also key predictors, correlating with consistently short beaks.
* Primary Lifestyle: The Terrestrial and Aerial categories are the strongest predictors overall. While the Aquatic lifestyle corresponds to the longest average beak length, it has very low feature importance compared to Terrestrial and Aerial lifestyles.
* Habitat: Aquatic habitat is a significant predictor within its category, correlating with longer beaks. 
* Migration: Sedentary behavior is the strongest predictor among migration behaviors, though overall importance for migration remains weak. 

## **5. Wing Size and Shape (Wing Length and Hand-Wing Index)**

### **5.1. Wing Length**

In [10]:
target = "Log_Wing"
predictors = ['Trophic.Niche', 'Primary.Lifestyle', 'Habitat', 'Migration']

X = pd.get_dummies(df_clean[predictors], drop_first=False)
y = df_clean[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42
)
rf.fit(X_train, y_train)
score = r2_score(y_test, rf.predict(X_test))

print("="*145)
print(f"Wing Length R² Score: {score:.3f}")
print("="*145)

display_columns = []
name_width = 21
val_width = 12

for col in predictors:
    grouped = df_clean.groupby(col)['Wing.Length'].mean().sort_values(ascending=False)

    formatted_rows = [f"{cat[:name_width]:<{name_width}} {val:>{val_width}.0f}" for cat, val in grouped.items()]
    display_columns.append(formatted_rows)

header_row = " | ".join([f"{col:<{name_width}} {'Length (mm)':>{val_width}}" for col in predictors])

print(header_row)
print("-" * len(header_row))

block_width = name_width + 1 + val_width

for row in zip_longest(*display_columns, fillvalue=" " * block_width):
    print(" | ".join(row))

print("="*145)

Wing Length R² Score: 0.453
Trophic.Niche          Length (mm) | Primary.Lifestyle      Length (mm) | Habitat                Length (mm) | Migration              Length (mm)
-------------------------------------------------------------------------------------------------------------------------------------------------
Scavenger                      536 | Aquatic                        228 | Aquatic                        200 | Partial                        153
Vertivore                      304 | Terrestrial                    148 | Open                           139 | Migratory                      144
Herbivore aquatic              255 | Aerial                         133 | Closed                         108 | Sedentary                      113
Herbivore terrestrial          230 | Generalist                     122 |                                    |                                   
Aquatic predator               215 | Insessorial                    104 |                       

In [11]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["Trophic Niche", "Primary Lifestyle", "Habitat", "Migration Behavior"],
    horizontal_spacing=0.15,
    vertical_spacing=0.15
)

importances = pd.Series(rf.feature_importances_, index=X.columns)

def add_panel(prefix, row, col, bar_color):
    subset = importances[importances.index.str.contains(prefix)].copy()

    subset.index = subset.index.str.replace(f"{prefix}_", "").str.replace(f"{prefix}.", "")
    
    subset = subset.sort_values(ascending=True)
    
    fig.add_trace(
        go.Bar(
            x=subset.values,
            y=subset.index,
            orientation='h',
            marker=dict(color=bar_color),
            name=prefix
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text="Importance (R²)", row=row, col=col)

add_panel("Trophic.Niche", 1, 1, "#BF616A")
add_panel("Primary.Lifestyle", 1, 2, "#8FBCBB")
add_panel("Habitat", 2, 1, "#EBCB8B")
add_panel("Migration", 2, 2, "#81A1C1")

fig.update_layout(
    title_text="Ecological Drivers of Wing Length",
    title_x=0.5,
    height=800,
    width=1100,
    showlegend=False,
    template="plotly_white"
)

fig.show()

The Random Forest model explains approximately 31% of the variation in avian beak length (R² = 0.311).

* Trophic Niche: A major predictor of beak length. Scavengers have the longest average beaks, but Aquatic predators are the strongest predictors (highest feature importance), likely due to the strict requirement for specialized bills to catch fish. Granivores are also key predictors, correlating with consistently short beaks.
* Primary Lifestyle: The Terrestrial and Aerial categories are the strongest predictors overall. While the Aquatic lifestyle corresponds to the longest average beak length, it has very low feature importance compared to Terrestrial and Aerial lifestyles.
* Habitat: Aquatic habitat is a significant predictor within its category, correlating with longer beaks. 
* Migration: Sedentary behavior is the strongest predictor among migration behaviors, though overall importance for migration remains weak. 

### **5.2. Wing Shape (Hand-Wing Index)**

In [12]:
target = "Log_HWI"
predictors = ['Trophic.Niche', 'Primary.Lifestyle', 'Habitat', 'Migration']

X = pd.get_dummies(df_clean[predictors], drop_first=False)
y = df_clean[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42
)
rf.fit(X_train, y_train)
score = r2_score(y_test, rf.predict(X_test))

print("="*145)
print(f"Hand-Wing Index R² Score: {score:.3f}")
print("="*145)

display_columns = []
name_width = 21
val_width = 12

for col in predictors:
    grouped = df_clean.groupby(col)['Hand-Wing.Index'].mean().sort_values(ascending=False)

    formatted_rows = [f"{cat[:name_width]:<{name_width}} {val:>{val_width}.0f}" for cat, val in grouped.items()]
    display_columns.append(formatted_rows)

header_row = " | ".join([f"{col:<{name_width}} {'HWI':>{val_width}}" for col in predictors])

print(header_row)
print("-" * len(header_row))

block_width = name_width + 1 + val_width

for row in zip_longest(*display_columns, fillvalue=" " * block_width):
    print(" | ".join(row))

print("="*145)

Hand-Wing Index R² Score: 0.505
Trophic.Niche                  HWI | Primary.Lifestyle              HWI | Habitat                        HWI | Migration                      HWI
-------------------------------------------------------------------------------------------------------------------------------------------------
Nectarivore                     50 | Aerial                          58 | Aquatic                         36 | Migratory                       35
Herbivore aquatic               43 | Aquatic                         41 | Open                            31 | Partial                         32
Aquatic predator                39 | Terrestrial                     25 | Closed                          23 | Sedentary                       23
Scavenger                       35 | Generalist                      22 |                                    |                                   
Herbivore terrestrial           34 | Insessorial                     21 |                   

In [13]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["Trophic Niche", "Primary Lifestyle", "Habitat", "Migration Behavior"],
    horizontal_spacing=0.15,
    vertical_spacing=0.15
)

importances = pd.Series(rf.feature_importances_, index=X.columns)

def add_panel(prefix, row, col, bar_color):
    subset = importances[importances.index.str.contains(prefix)].copy()

    subset.index = subset.index.str.replace(f"{prefix}_", "").str.replace(f"{prefix}.", "")
    
    subset = subset.sort_values(ascending=True)
    
    fig.add_trace(
        go.Bar(
            x=subset.values,
            y=subset.index,
            orientation='h',
            marker=dict(color=bar_color),
            name=prefix
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text="Importance (R²)", row=row, col=col)

add_panel("Trophic.Niche", 1, 1, "#BF616A")
add_panel("Primary.Lifestyle", 1, 2, "#8FBCBB")
add_panel("Habitat", 2, 1, "#EBCB8B")
add_panel("Migration", 2, 2, "#81A1C1")

fig.update_layout(
    title_text="Ecological Drivers of Hand-Wing Index",
    title_x=0.5,
    height=800,
    width=1100,
    showlegend=False,
    template="plotly_white"
)

fig.show()

The Random Forest model explains approximately 49% of the variation in the Hand-Wing Index (R² = 0.49), which is a proxy for wing pointedness and dispersal ability.

* Trophic Niche: A major predictor of wing shape. Nectarivores have the highest average HWI, but Invertivores (insect eaters) are the strongest predictors.
* Primary Lifestyle: The Aerial lifestyle is the single strongest predictor overall and corresponds to the highest average HWI.
* Habitat: Closed (forest) habitat is a significant predictor within its category, correlating with lower HWI. 
* Migration: Sedentary behavior is the strongest predictor among migration behaviors, corresponding to lower HWI compared to migratory species.